In [146]:
import os
import xml.etree.ElementTree as ET
if os.getcwd!='C:\\Users\\Alice\\Dropbox (MIT)\\smartRhythm\\musicxmlset':
    os.chdir('C:\\Users\\Alice\\Dropbox (MIT)\\smartRhythm\\musicxmlset')
import pandas

In [ ]:
class NotePosition():
    def __init__(self, noteNum, absolutePos):
        self.noteNum = noteNum
        self.absolutePos = absolutePos
    def __hash__(self):
        return hash(str(self.noteNum)+str(self.absolutePos))
    def __str__(self):
        return str(self.noteNum) + "-" + str(self.absolutePos) + "."
    def __repr__(self):
        return str(self.noteNum) + "-" + str(self.absolutePos) + "."
    def __eq__(self, other):
        if isinstance(other, NotePosition):
            return self.noteNum == other.noteNum and self.absolutePos == other.absolutePos
        return False
    def __ne__(self, other):
        return not(self == other)
    def __lt__(self,other):
        if self.absolutePos !=None and other.absolutePos!=None:
                return self.absolutePos < other.absolutePos
        else:
                return self.noteNum < other.noteNum            
    def __gt__(self, other):        
        if self.absolutePos !=None and other.absolutePos!=None:
            return self.absolutePos < other.absolutePos
        else:
            return self.noteNum < other.noteNum    

In [ ]:
class NoteStats():
    def addMeasure(self, durationLists, divisions, timesignum, timesigden):
        ## for each list in durationlists:
            ##divide durations by divisions or something
            ##add list to notestats for the timesigstuff 

In [147]:
def getDurationLists(staffDict):
    allLists = []
    for staff in staffDict:
        measureNotes = staffDict[staff].items()
        print measureNotes
        sortedNotes=sorted(measureNotes, key=lambda measure: measure[0])
        allLists.append([x[1] for x in sortedNotes])
    return allLists

In [ ]:
def checkDurationLists(durationLists, division, timesignum):
    for l in durationLists:
        totalDuration = sum(l)
        if not division * timesignum ==totalDuration:
            print "division", division, "timesignum", timesignum, "totalDuration", totalDuration
            print division * timesignum
            print "NOT CORRECT NUMBER OF BEATS"
        durationLists.remove(l)

In [148]:
def positionOverlap(durationList, xpos):
    if xpos==None:
        return None
    for notePosition in durationList:
        if xpos== notePosition.absolutePos:
            return notePosition
    return None

def parseMusicXMLFile(file):
    tree = ET.parse(file)
    root = tree.getroot()
    part=root.find('part')
    timesignum=0
    timesigden=0
    divisions=0    
    for part in root.findall('part'):
        for attribute in part.find('measure').findall('attributes'):
            time = attribute.find('time')
            if time is not None:
                timesignum = int(time.find('beats').text)
                timesigden = int(time.find('beat-type').text)
            div = attribute.find('divisions')
            if div is not None:
                divisions=int(div.text)
        if timesignum!=4 or timesigden!=4:
            return
        assert divisions is not None
        for measure in part.findall('measure'):
            measurestr = ""
            measureLength = 0
            ##staffdict is a dictionary {staffNumber:{NotePosition, duration}}
            staffdict = {}
            noteNum = 1
            for note in measure.findall('note'):
                if note.find('grace') is not None:
                    print "grace note"
                    break
                duration = int(note.find('duration').text)
                staff = 0
                voice = -1
                if note.find('staff') is not None:
                    staff = note.find('staff').text
                if note.find('voice') is not None:
                    voice = note.find('voice').text
                ##ensures each note gets a unique key in the dictionary, by default
                xpos = None
                ##If the note has an x-position, note it
                if 'default-x' in note.attrib:
                    xpos = float(note.attrib['default-x'])

                if voice in staffdict:
                    overlap = positionOverlap(staffdict[voice], xpos)
                    if overlap!=None:
                        ##a note has already been counted at this same position. 
                        ##Take the shorter note, so that we don't overcount anything
                        if staffdict[voice][overlap]>duration:
                            del staffdict[voice][overlap]
                            staffdict[voice][NotePosition(noteNum,xpos)] = duration                    
                    else:
                        staffdict[voice][NotePosition(noteNum,xpos)]=duration
                else:
                    noteNum = 1
                    staffdict[voice] = {NotePosition(noteNum,xpos):duration}
                noteNum +=1
            durationLists = getDurationLists(staffdict)
            print "durationlist", durationLists
            checkDurationLists(durationLists, divisions, timesignum)
            addMeasure(durationLists, divisions, timesignum, timesigden)
        print "next part"

In [149]:
noteStats = NoteStats()
files=os.listdir('.')
durations=[]
for file in files:
    print "for file " + file
    parseMusicXMLFile(file)
##parseMusicXMLFile(files[0])

for file Aires de Rumba.xml
for file Beethoven-Grosse Fuge.xml
for file Catro Vellos Mari�eiros.xml
for file COSTELEY Je vois de glissantes eaux.xml
for file D Blues.xml
for file Da_farfar_var_ung-akk.xml
for file Deck_the_Halls.xml
[(3-180.79., 2), (1-None., 3), (2-149.84., 1), (4-230.31., 2)]
[(1-None., 4), (2-180.79., 3), (3-261.26., 1)]
durationlist [[3, 1, 2, 2], [4, 3, 1]]
[(4-164.04., 2), (1-20.4., 2), (2-68.28., 2), (3-116.16., 2)]
[(1-20.4., 2), (2-68.28., 2), (3-116.16., 4)]
durationlist [[2, 2, 2, 2], [2, 2, 4]]
[(3-86.4., 1), (4-119.4., 1), (2-53.4., 1), (1-20.4., 1), (7-251.4., 1), (5-152.4., 3)]
[(3-152.4., 1), (5-218.4., 1), (6-251.4., 1), (4-185.4., 1), (1-None., 2), (2-86.4., 2)]
durationlist [[1, 1, 1, 1, 3, 1], [2, 2, 1, 1, 1, 1]]
[(2-69.45., 2), (1-20.4., 2), (3-118.51., 4)]
[(2-69.45., 2), (1-20.4., 2), (3-118.51., 4)]
durationlist [[2, 2, 4], [2, 2, 4]]
[(2-129.56., 1), (4-190.79., 2), (1-68.33., 3), (3-153.11., 2)]
[(4-214.34., 1), (1-68.33., 2), (2-106.01., 2), 

In [150]:
os.getcwd()

'C:\\Users\\Alice\\Dropbox (MIT)\\smartRhythm\\musicxmlset'

In [151]:
None==None

True

In [152]:
class Test():
    def __str__(self):
        return "b"
    def __repr__(self):
        return "a"

x= Test()
print x

b
